So I tried to do emkhes algorithm on 4 nodes

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import math
import copy

Import data

In [ ]:
def import_data():
    #locate it to my dummy data
    gtfs_dir = 'C:/Users/Diana Lutska/DAP/gtfs_dummy_data/'

    # load data
    agency_df = pd.read_csv(gtfs_dir + 'agency.txt')
    stops_df = pd.read_csv(gtfs_dir + 'stops.txt')
    routes_df = pd.read_csv(gtfs_dir + 'routes.txt')
    trips_df = pd.read_csv(gtfs_dir + 'trips.txt')
    stop_times_df = pd.read_csv(gtfs_dir + 'stop_times.txt')
    calendar_df = pd.read_csv(gtfs_dir + 'calendar.txt')

    return agency_df, stops_df, routes_df, trips_df, stop_times_df, calendar_df

agency_df, stops_df, routes_df, trips_df, stop_times_df, calendar_df = import_data()

Creating legs

In [ ]:
# Sort by trip_id and stop_sequence to ensure legs are in order
legcreation_df = stop_times_df.sort_values(['trip_id', 'stop_sequence']).reset_index(drop=True)


#Shift the columns to get the arrival stop and time for each departure stop and time
legcreation_df['arrival_stop_id'] = legcreation_df['stop_id'].shift(-1)
legcreation_df['arrival_time'] = legcreation_df['arrival_time'].shift(-1)

print(legcreation_df)

    trip_id arrival_time  ... stop_sequence  arrival_stop_id
0        11     06:03:00  ...             1              2.0
1        11     06:05:00  ...             2              1.0
2        12     06:08:00  ...             1              2.0
3        12     06:03:00  ...             2              2.0
4        21     06:08:00  ...             1              4.0
5        21     06:08:00  ...             2              2.0
6        22     06:13:00  ...             1              4.0
7        22     06:00:00  ...             2              1.0
8        31     06:04:00  ...             1              3.0
9        31     06:04:00  ...             2              1.0
10       32     06:07:00  ...             1              3.0
11       32     06:04:00  ...             2              3.0
12       41     06:10:00  ...             1              4.0
13       41     06:05:00  ...             2              3.0
14       42     06:11:00  ...             1              4.0
15       42         None

In [ ]:
# Filter out the last stop of each trip, as it has no next stop
legs_df = legcreation_df[legcreation_df['trip_id'] == legcreation_df['trip_id'].shift(-1)].copy()
legs_df['arrival_stop_id']=legs_df['arrival_stop_id'].astype(int)

print(legs_df.head())

   trip_id arrival_time departure_time  stop_id  stop_sequence  arrival_stop_id
0       11     06:03:00       06:00:30        1              1                2
2       12     06:08:00       06:05:30        1              1                2
4       21     06:08:00       06:03:30        2              1                4
6       22     06:13:00       06:08:30        2              1                4
8       31     06:04:00       06:00:30        1              1                3


In [ ]:
# Create "route_id" column
legs_df = legs_df.merge(trips_df, on='trip_id', how='left')
print(legs_df)

   trip_id arrival_time departure_time  ...  arrival_stop_id  route_id  service_id
0       11     06:03:00       06:00:30  ...                2         1           1
1       12     06:08:00       06:05:30  ...                2         1           1
2       21     06:08:00       06:03:30  ...                4         2           1
3       22     06:13:00       06:08:30  ...                4         2           1
4       31     06:04:00       06:00:30  ...                3         3           1
5       32     06:07:00       06:04:30  ...                3         3           1
6       41     06:10:00       06:04:30  ...                4         4           1
7       42     06:11:00       06:05:30  ...                4         4           1

[8 rows x 8 columns]


In [ ]:
#Creating legs tuples
legs_df['leg'] = list(zip(
    zip(legs_df['stop_id'], legs_df['departure_time']),
    zip(legs_df['arrival_stop_id'], legs_df['arrival_time']),
    legs_df['route_id']
))

legs_df = legs_df[['trip_id', 'leg']]
print(legs_df)

   trip_id                                leg
0       11  ((1, 06:00:30), (2, 06:03:00), 1)
1       12  ((1, 06:05:30), (2, 06:08:00), 1)
2       21  ((2, 06:03:30), (4, 06:08:00), 2)
3       22  ((2, 06:08:30), (4, 06:13:00), 2)
4       31  ((1, 06:00:30), (3, 06:04:00), 3)
5       32  ((1, 06:04:30), (3, 06:07:00), 3)
6       41  ((3, 06:04:30), (4, 06:10:00), 4)
7       42  ((3, 06:05:30), (4, 06:11:00), 4)


Reliabilty function

In [ ]:
# Function to calculate probability of successful transfer between two subsequent legs
def calculate_transfer_probability(prev_leg: pd.Series, next_leg: pd.Series) -> float:

    #If line = next line , then probability = 1 since the traveler remains on the same line
    if prev_leg[1][2] == next_leg[1][2]:
        return 1  
    else:
        #check if departure time is larger than arrival
        if prev_leg[1][1][1] > next_leg[1][0][1]:
            return 0
        else :
        # created random probability, just for the test
            return 0.5
            

In [ ]:
# i hope it is cumulative distribution
def calculate_cumulative_probability(itinerary) -> list[float]:
    cumulative_probabilities = [1] 
    for i in range(len(itinerary) - 1):
        prev_leg = itinerary[i]
        next_leg = itinerary[i+1]

        transfer_prob = calculate_transfer_probability(prev_leg, next_leg)
        cumulative_probabilities.append(cumulative_probabilities[-1] * transfer_prob)
    return cumulative_probabilities
  

In [ ]:
#conditional probability of making all previous transfers between each leg 1 . . . m of the itinerary. 
def calculate_product_of_probabilities(cumulative_probabilities: list[float]) -> float:
        product_of_probabilities = math.prod(cumulative_probabilities)
        return product_of_probabilities


In [ ]:
# check 
itinerary = [legs_df.iloc[0],legs_df.iloc[2]]
print(itinerary)
cumulative_probabilities = calculate_cumulative_probability(itinerary)
#print(cumulative_probabilities)
product = calculate_product_of_probabilities(cumulative_probabilities)
print(product)

[trip_id                                   11
leg        ((1, 06:00:30), (2, 06:03:00), 1)
Name: 0, dtype: object, trip_id                                   21
leg        ((2, 06:03:30), (4, 06:08:00), 2)
Name: 2, dtype: object]
0.5


Probability of arrival on time

In [ ]:
# if there is a chance to have all succesful transfers, is the arrival time in time budget
def calculate_arrival_probability(itinerary, start_time, time_budget) -> int:
    # Check if all previous transfers were successful
    if math.prod(calculate_cumulative_probability(itinerary)) > 0:  # Only proceed if all transfers are successful( naka can be possibly made)
        # Calculate the actual arrival time at the final leg
        destination_leg = itinerary[-1]
        destination_arrival_time = destination_leg[1][1][1]
        
        start_time = datetime.strptime(start_time, "%H:%M:%S")
        destination_arrival_time = datetime.strptime(destination_arrival_time, "%H:%M:%S")

        total_travel_time = destination_arrival_time - start_time

        # Check if total travel time is within the budget
        if total_travel_time <= time_budget:
            return 1  # Probability of 1 if arrival is within the time budget
        else:
            return 0  # Probability of 0 if arrival is beyond the time budget
    else:
        return 0  # Probability of 0 if any prior transfer was unsuccessful
    

In [ ]:
start_time = "06:00:00"
time_budget = timedelta(hours=0, minutes=10)
a = calculate_arrival_probability(itinerary,start_time,time_budget)
print(a)

1


In [ ]:
def primary_itinerary_reliability(itinerary, start_time, time_budget) -> float:
    
    cumulative_probabilities = calculate_cumulative_probability(itinerary)
    product_of_probabilities= math.prod(cumulative_probabilities)
    arrival_probability = calculate_arrival_probability(itinerary,start_time,time_budget)

    reliability = arrival_probability * product_of_probabilities
    return reliability

In [ ]:
# check if based on three paramethers
itinerary = [legs_df.iloc[4],legs_df.iloc[6]]
start_time = "06:00:00"
time_budget = timedelta(hours=0, minutes=12)
reliability = primary_itinerary_reliability(itinerary,start_time,time_budget)
print(reliability)

0.5


In [ ]:
backup_itinerary = [legs_df.iloc[7]]
print(backup_itinerary[0][1][0][0])


3


In [ ]:
#im not sure how the "backup" will look like (like is it starting from the origin or from the missed transfer point)
# backup itineary== (leg of the prim it where is transfer, [sequence of legs of backup starting from the next after transfer], reliability, arrival time)
# but for now, backup itineary = [sequence of legs]
def backup_itinerary_reliability(itinerary, backup_itinerary:list[pd.Series], start_time, time_budget) -> float:

    arrival_probability = calculate_arrival_probability(backup_itinerary, start_time, time_budget)
    cumulative_probabilities = calculate_cumulative_probability(itinerary)
    product_of_probabilities= math.prod(cumulative_probabilities)
    
    transfer_point = backup_itinerary[0][1][0][0]
    initial_transfer_prob = 1 # Default value in case no missed transfer is identified

    for idx, leg in enumerate(itinerary[:-1]):  # Exclude last item since we're accessing i+1
        if leg[1][1][0] == transfer_point:
            prev_leg = itinerary[idx]
            missed_leg = itinerary[idx + 1]
            initial_transfer_prob = calculate_transfer_probability(prev_leg,missed_leg)
            break

    backup_reliability = arrival_probability * product_of_probabilities * (1 - initial_transfer_prob)
    
    return backup_reliability

In [ ]:
a = backup_itinerary_reliability(itinerary, backup_itinerary,start_time,time_budget)
print(a)

0.25


In [ ]:
#Reliability of a complete itinerary 
#Backups will be set of backups of prim itineary
# backup itineary= (leg of the prim it where is transfer, [sequence of legs of backup starting from the next after transfer], reliability, arrival time)
def itinerary_reliability(itinerary : list[pd.Series],Backups : list[tuple], start_time:str, time_budget: timedelta) -> float:
    primary_reliability = primary_itinerary_reliability(itinerary,start_time,time_budget)
    added_reliability = 0
    for b in Backups:
        #this will be fucked up, as we dont know yet how to make backups ( like on what format)
        # i used Set for all backups, but set dont allow df nor lists, so backup_sequence is turple)
        backup_itinerary = b[1]
        # Convert indices back to full rows using legs_df
        backup_reliability = backup_itinerary_reliability(itinerary,backup_itinerary,start_time,time_budget)
        added_reliability += backup_reliability
    

    complete_reliability = primary_reliability + added_reliability
    return complete_reliability

Check of this whole reliability thing

In [ ]:
#check of this whole reliability bullshit
itinerary = [legs_df.iloc[4],legs_df.iloc[6]]
start_time = "06:00:00"
time_budget = timedelta(hours=0, minutes=12)

# backup itineary= (leg of the prim it where is transfer, [sequence of legs of backup starting from the next after transfer], reliability, arrival time)
# idk whats happening at this point actually

transfer_leg = itinerary[0]

backup_itinerary = [legs_df.iloc[7]]
backup_reliability = backup_itinerary_reliability(itinerary, backup_itinerary,start_time,time_budget)

destination_leg = backup_itinerary[-1]
arrival_time = destination_leg[1][1][1]

backup1 = (transfer_leg,backup_itinerary,backup_reliability,arrival_time)
#creating backups list for trying out (too much hasstle with set)
Backups = [backup1]

a = itinerary_reliability(itinerary,Backups,start_time,time_budget)
print(a)

0.75


We have now reliability function that calculates reliability for the complete itinerary. Now we have to create algorithm of search of the itineary...

Network search algorithm

In [ ]:
#not sure if here we should add time dependancy
def search_adjecent_legs(node_id):
    filtered_legs = []
    for index, row in legs_df.iterrows():
        if row[1][0][0] == node_id:  # leg[0][0] corresponds to stop_id in the tuple (stop_id, departure_time)
            filtered_legs.append(row)
    return filtered_legs

In [ ]:
# need to add to arrival probability
def travel_time(itinerary, start_time):
    destination_leg = itinerary[-1]
    destination_arrival_time = destination_leg[1][1][1]
        
    start_time = datetime.strptime(start_time, "%H:%M:%S")
    destination_arrival_time = datetime.strptime(destination_arrival_time, "%H:%M:%S")

    total_travel_time = destination_arrival_time - start_time
    return total_travel_time

In [ ]:
def is_transfer(itinerary) -> bool:
    prev_leg = itinerary[-2]
    next_leg = itinerary[-1]
    if prev_leg[1][2] == next_leg[1][2]:
        return False
    else:
        return True

Ehmke's algorythm ( almost)

In [ ]:
def find_path(origin_node: int,destination_node: int, start_time : str, time_budget: timedelta):
    MRIB_reliability = 0
    MRIB = None
    LIST = []

    adjecent_legs = search_adjecent_legs(origin_node)

    for leg in adjecent_legs:
        itinerary  = [leg]
        reliability = 1
        duration = travel_time(itinerary,start_time)
        expected_arrival_time = leg[1][1][1]
        backups= []
        LIST.append([itinerary,reliability,duration,expected_arrival_time,backups])
    
        
    while len(LIST) > 0:
        # Find the shortest itinerary
        shortest_path = min(LIST, key=lambda x: x[2])  # x[2] is duration (timedelta)
        #print(shortest_path)
        
        #will be issues with that laterrrrr 100%

        LIST = [item for item in LIST if item[2] != shortest_path[2]]
        #print(len(LIST))
        #going further into the network

        tail = shortest_path[0][-1]
        if tail[1][1][0] == destination_node:
            rel = itinerary_reliability(shortest_path[0],backups,start_time,time_budget)
            if rel >MRIB_reliability:
                MRIB_reliability = rel
                MRIB = shortest_path
        else:
            #here will be all possible path from the second node
            LIST1 = []
            #set of all legs adjacent to tail leg m
            next_legs = search_adjecent_legs(tail[1][1][0])
            #print(next_legs)
            for leg in next_legs:
                itinerary = shortest_path[0] + [leg]
                #print(itinerary)
                reliability = itinerary_reliability(itinerary,Backups,start_time,time_budget)
                #print(reliability)
                duration = travel_time(itinerary,start_time)
                #print(duration)
                expected_arrival_time = leg[1][1][1]
                #print(expected_arrival_time)
                backup = []
                if reliability > 0: # check initially so we dont waste time later
                    LIST1.append([itinerary,reliability,duration,expected_arrival_time,backup])
                # If leg l j in set of transfer legs T i 
            #print(LIST1)
            if len(LIST1) > 0:
                shortest_next_itinerary = min(LIST1, key=lambda x: x[2])
                
                if is_transfer(shortest_next_itinerary[0]) is False:
                    LIST.append(shortest_next_itinerary)
                else :
                    #remove transfer so only backups are left
                    LIST1.remove(shortest_next_itinerary)
                    if len(LIST1) > 0:
                        shortest_backup_path = min(LIST1, key=lambda x: x[2])
                        shortest_backup = shortest_backup_path[0][-2:]
                        #print("a",shortest_backup)
                        tail = shortest_backup[-1]
                        if tail[1][1][0] == destination_id:
                            backup = (shortest_backup[0],[shortest_backup[1]])
                            #print("b",backup)
                            #print(shortest_next_itinerary)
                            shortest_next_itinerary[4].append(backup)
                            #print("aaa",shortest_next_itinerary)
                            LIST.append(shortest_next_itinerary)
                        else:
                            print("well i need to think about that, some round function idk")
                    else:
                        print("no backups")
            else:
                print("no further feasible connections")
    return MRIB_reliability, MRIB
        


Okay now we can checkkk

In [ ]:
origin = 1 # for now the departure node is 1
destination_id = 4
start_time = "06:00:00"
time_budget = timedelta(hours=0, minutes=15)

a,b = find_path(origin,destination_id,start_time,time_budget)
print(a,b)

no further feasible connections
no backups
0.5 [[trip_id                                   31
leg        ((1, 06:00:30), (3, 06:04:00), 3)
Name: 4, dtype: object, trip_id                                   41
leg        ((3, 06:04:30), (4, 06:10:00), 4)
Name: 6, dtype: object], 0.75, datetime.timedelta(seconds=600), '06:10:00', [(trip_id                                   31
leg        ((1, 06:00:30), (3, 06:04:00), 3)
Name: 4, dtype: object, [trip_id                                   42
leg        ((3, 06:05:30), (4, 06:11:00), 4)
Name: 7, dtype: object])]]


Well this is correct, but now we need to see if it works only on this data (ofc it will not work on OBB data without adj) !! ACTUALLY IT IS NOT